In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/projects/proj

/content/gdrive/MyDrive/projects/proj


In [134]:
import sys
sys.path.append('/content/gdrive/MyDrive/Models')

In [136]:
from autoencoder import aphl
from deepmimo import Deep_MIMO
from mist import MIST
from pe import PE

In [138]:
a1 = aphl("/content/gdrive/MyDrive/projects/RML2016.10a/")
p1 = PE("/content/gdrive/MyDrive/projects/project_3/PE")
mim1 = Deep_MIMO('/content/gdrive/MyDrive/projects/DeepMIMO2',active_BS=20,user_row_last=2)
mis1 = MIST()

In [142]:
import subprocess
subprocess.call(['pip', 'install', "DeepMIMO"])

0

## autoencoder 

In [46]:
from generate_model import Designed_model as GM
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Reshape,Dense, GaussianNoise,Lambda,Dropout
from keras.models import Model
from keras import regularizers
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

class aphl(GM):
   def __init__(self,data_path):
      super(aphl, self).__init__(data_path)
      self.name ='aphl'
   
   def build_model_and_get_data(self):
      if type(self.X_train) == type(None) : self.set_data() 
      self.model = self.CNN_model_article(
         self.X_train[0].shape, 
         self.y_train[0].shape[0])
      self.compile_model(self.model)
      return 
   
   def train_gpu(self,epochs =20,batch_size=1024 ,compile = True): 
       with tf.device(tf.test.gpu_device_name()):
          self.train(epochs=epochs, batch_size=batch_size)    

   def train(self, epochs =20,batch_size=1024 ,compile = True, get_data_build =True):
      filepath = self.data_path
      if get_data_build : self.build_model_and_get_data()
      if compile : self.model = self.compile_model(self.model)
      history = self.model.fit(self.X_train,self.y_train,
              validation_data=(self.X_test, self.y_test),
              callbacks = [
                    keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
                    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')],
              epochs=epochs, batch_size = batch_size, verbose=2 )
      
      return history
   
   
   def set_data(self , limit = None) -> None:
      ##read from directory
      df = pd.read_pickle( self.data_path + "RML2016.10a_dict.pkl")
      d = pd.DataFrame(df.keys())
      snrs,mods = d[0].unique() , d[1].unique().tolist()
      if type (snrs[1]) == str : 
         snrs,mods = d[1].unique() , d[0].unique().tolist()
      ## creat label 
      X = []  
      label = []
      for key , value in df.items():
         label.extend([key]*value.shape[0])
         X.append(value)
      X = np.vstack(X)
      
      ## make one-hot label
      encoded = to_categorical([i for i in range(len(mods))]) 
      
      # seperate data for train and test
      y = label 
      
      if self.limit_check(limit,X):
            X = X[:limit]
            y = X[:limit] 

      X_mean = self.mean_data(X)
      self._mean = X_mean
      
      X_scale = self.scale(X)
      self._mean = X_scale
      
      X = self.transform(X,X_mean,X_scale)

      X_train, X_test, y_train, y_test = train_test_split(
                                             X, y, test_size=0.4, random_state=42)

      SNR_train = np.array([lbl[1] for lbl in y_train])
      y_train = np.array([self.mods_to_one_hot(mods,lbl[0],encoded) for lbl in y_train])
      y_test  = np.array([self.mods_to_one_hot(mods,lbl[0],encoded) for lbl in y_test])

      sorted_arg_SNR_descended = SNR_train.argsort()[::-1]
      X_train = X_train[sorted_arg_SNR_descended]
      y_train = y_train[sorted_arg_SNR_descended]

      X = X.astype('float64')
      y_train = y_train.astype('float64')
      y_test = y_test.astype('float64')

      self.X_train , self.y_train, self.X_test, self.y_test = X_train ,y_train, X_test,y_test
   
   @staticmethod
   def compile_model(model):
      model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=0.001),
                    loss= tf.keras.losses.CategoricalCrossentropy(),
                    metrics=["accuracy"],
                )  
      return model
   
   @staticmethod
   def mods_to_one_hot (mods,inp,encoded):
     
    return encoded[mods.index(inp)]


   @staticmethod
   def complex_to_array(dataset):
      shape = dataset.shape
      temp = dataset.reshape([*shape,1])
      real_num , imag_num = np.real(temp) , np.imag(temp)
      array_form_of_complex = np.concatenate((real_num,imag_num),axis=len(shape))
      return array_form_of_complex 

   @staticmethod
   def transform(x,mean,scale):
     return (x-mean)/scale


   @staticmethod 
   def mean_data(data):
    for i in range(len(data.shape)-2):
        if i == 0 : mean = np.mean(data,axis=0)
        mean = np.mean(mean,axis=0)   
    return mean
    
   @staticmethod 
   def scale(data):
    max = np.max(np.abs(data))
    return  max 

   
   @staticmethod
   def CNN_model_article(input_shape = (2,128) , output_shape=11):

    input_signal = Input(shape = input_shape)
    reshape = Reshape((2,128,1))(input_signal)
    conv_1 = Conv2D(128 , kernel_size = (2, 8), activation='relu' )(reshape)
    max_pool_1 = MaxPooling2D (pool_size=(1,2), strides = 2,data_format ='channels_last')(conv_1)
    
    conv_2 = Conv2D(64, kernel_size =(1, 16), activation='relu')(max_pool_1)
    max_pool_2 = MaxPooling2D (pool_size=(1,2), strides = 2)(conv_2)

    flatten = Flatten()(max_pool_2)

    dense_1 = Dense(128, activation='relu')(flatten)
    dense_2 = Dense(64, activation='relu')(dense_1)
    dense_3 = Dense(32, activation='relu')(dense_2)
    dense_4 = Dense(output_shape, activation='softmax')(dense_3)

    return Model(inputs = input_signal, outputs = dense_4)  

## DeepMIMO

In [5]:
!pip install DeepMIMO

In [6]:
import DeepMIMO
import numpy as np 

In [97]:
import DeepMIMO
import numpy as np 
from generate_model import Designed_model as GM
from sklearn.model_selection import train_test_split
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers , Input , Sequential 
from tensorflow.keras.layers import Flatten , Dense , Reshape 
import keras.backend as K


class Deep_MIMO(GM):
   def __init__(self,data_path ,active_BS = 64,
                                user_row_first = 1,
                                user_row_last = 10,
                                bandwidth = 0.05,
                                subcarriers = 64 ,
                                subcarriers_sampling = 1 ,
                                subcarriers_limit = 32,
                                num_paths = 1,
                                num_set_antenna_M = 8):
      super(Deep_MIMO, self).__init__(data_path)
      for i,(key,value) in enumerate(locals().items()):
          if i < 2 : continue
          setattr(self,key,value)
      self.name ="deepmimo"

   
   def build_model_and_get_data(self,MLP_DroupOut = True):
      if type (self.X_train) == type(None): self.set_data() 

      input_shape = self.X_train[0].shape
      output_shape = self.y_train[0].shape

      if MLP_DroupOut : self.model = self.MLP_with_dropouts (input_shape , output_shape)
      else : self.model = self.MLP( input_shape , output_shape)
      self.compile_model (self.model, self.MSE_loss )
      
      return self.model
   

   def train_gpu(self,epochs =17,batch_size=100 ,compile = True ,get_data_build =True) -> None: 
       with tf.device(tf.test.gpu_device_name()):
          self.train(epochs=epochs, batch_size=batch_size , compile= compile, )    

   def train(self, epochs =17,batch_size=100 ,compile = True, get_data_build =True) -> None:
      if get_data_build : self.build_model_and_get_data()
      if compile : self.model = self.compile_model (self.model, self.MSE_loss )
      history = self.model.fit(self.X_train,self.y_train,
              validation_data=(self.X_test, self.y_test),
              epochs=epochs, batch_size = batch_size, verbose=2 )
      return history
   
   
   def set_data(self , limit = None) -> None:

      dataset_up = self.get_MIMO_data('I1_2p4')
      dataset_down = self.get_MIMO_data('I1_2p4')

      dataset_up = self.draw_necessary_inf_in_desired_shape (dataset_up)
      dataset_down = self.draw_necessary_inf_in_desired_shape (dataset_down)

      data_up = self.complex_to_array(dataset_up)
      data_down = self.complex_to_array(dataset_down)

      
      X_mean = self.mean_data(data_up)
      y_mean = self.mean_data(data_down)
      self._mean = X_mean
      
      X_scale = self.scale(data_up)
      y_scale = self.scale (data_down)
      self._scale = X_scale
      
      norm_data_up = self.transform(data_up,X_mean,X_scale)
      norm_data_down = self.transform(data_down,y_mean,y_scale)

      X = np.array([u*self.make_mask(u,num_set_antenna_M = self.num_set_antenna_M) for u in norm_data_up[:]])
      y = norm_data_down

      X = X.astype('float64')
      y = y.astype('float64')
      X_train, X_test, y_train, y_test = train_test_split(
                                              X, y, test_size=0.33, random_state=42)
      
      self.X_train , self.y_train, self.X_test, self.y_test = X_train ,y_train, X_test,y_test


   
   def compile_model(self, model, func ):
      model.compile(
                      optimizer=keras.optimizers.Adam(learning_rate=0.001),
                      loss= func,
                      metrics=["accuracy"],
                  )  
      return model  

   def model_eval (batch_size = 100):
      self.model.evaluate(self.X_test, self.y_test, batch_size=batch_size, verbose=2)

   
   @staticmethod
   def draw_necessary_inf_in_desired_shape(data):
     #just using one RX antenna data and one TX antenna form all antennas information  
      data = [data[i]['user']["channel"][:][:,0,0] for i in range(len(data))]
      data = np.array(data)
      print(data.shape)
      #changing the data into form of : [num_user,num_antena,num_subchannel]  
      data = np.transpose(data,axes=(1,0,2))
      return data
   
   @staticmethod
   def mods_to_one_hot (mods,inp,encoded):
     
    return encoded[mods.index(inp)]
   
   @staticmethod
   def MLP(input_shape,output_shape,
                      num_layers = [2**10,2**12,2**12,64*32*2],
                      k_r = None):
     
      num_last_layer = output_shape[0]*output_shape[1]*output_shape[2]
      input_signal = Input(shape = input_shape) 
      flatten =Flatten()(input_signal)
      for i,num in enumerate(num_layers[1:-1]):
        if i==0: dense = Dense(num, activation="relu" , kernel_regularizer = k_r)(flatten)
        dense = Dense(num, activation="relu" , kernel_regularizer = k_r)(dense)

      last_dense = Dense(num_last_layer, activation=None , kernel_regularizer = k_r)(dense)    
      out_put = Reshape(output_shape, input_shape=(last_dense))(last_dense)  
      return Model(inputs=input_signal,outputs=out_put) 
 

   @staticmethod
   def MLP_with_dropouts(input_shape,output_shape,
                      num_layers = [2**10,2**12,2**12,64*32*2],
                      dropout_rate=.1 ,
                      k_r = None):
     
      num_last_layer = output_shape[0]*output_shape[1]*output_shape[2]
      input_signal = Input(shape = input_shape) 
      flatten =Flatten()(input_signal)
      for i,num in enumerate(num_layers[1:-1]):
        if i==0: dense = Dense(num, activation="relu" , kernel_regularizer = k_r)(flatten)
        dense = Dense(num, activation="relu" , kernel_regularizer = k_r)(dense)
        drop = Dropout(dropout_rate)(dense)

      last_dense = Dense(num_last_layer, activation=None , kernel_regularizer = k_r)(drop)    
      out_put = Reshape(output_shape, input_shape=(num_last_layer,))(last_dense)   
      return Model(inputs=input_signal,outputs=out_put) 


   @staticmethod
   def complex_to_array(dataset):
      shape = dataset.shape
      temp = dataset.reshape([*shape,1])
      real_num , imag_num = np.real(temp) , np.imag(temp)
      array_form_of_complex = np.concatenate((real_num,imag_num),axis=len(shape))
      return array_form_of_complex 

   @staticmethod
   def transform(x,mean,scale):
     return (x-mean)/scale


   @staticmethod 
   def mean_data(data):
    for i in range(len(data.shape)-2):
        if i == 0 : mean = np.mean(data,axis=0)
        mean = np.mean(mean,axis=0)   
    return mean
    
   @staticmethod 
   def scale(data):
    max = np.max(np.abs(data))
    return  max 


   @staticmethod  
   def make_mask (dataset , inp_mask = None , num_set_antenna_M = 8):
    num_ant ,num_subchannel = dataset.shape[0:2];
    mask = np.zeros(dataset.shape);

    if inp_mask == None : 
        antena_selected_mask = np.random.choice(num_ant,
                                                num_set_antenna_M,
                                                replace=False)
        
    else: antena_selected_mask = np.where( mask == 1)
    mask[ antena_selected_mask] =  1
    return mask

   @staticmethod
   def plot_imshow(*atr):
    fig = make_subplots(rows=len(atr), cols=2)
    for j in range(len(atr)):
        for i in range(2):
            fig.add_trace(px.imshow(atr[j][:,:,i]).data[0], row=j+1,col=i+1)
    fig.show()


   @staticmethod
   def NMSE_loss (y_true , y_pred):
    return tf.cast(K.mean(K.square(y_pred - y_true)/ (2*K.square(y_true))),tf.float64)

   @staticmethod
   def MSE_loss (y_true , y_pred):
    return K.mean(K.square(y_pred-y_true))

  
   def get_MIMO_data(self,scenario ) :

     # Load the default parameters
      parameters = DeepMIMO.default_params()
      # Set scenario name
      parameters['scenario'] = scenario
      # Set the main folder containing extracted scenarios
      parameters['dataset_folder'] = r''+ self.data_path
      # To activate the basestations from 1 tiil 64, set
      # each antena has 8 RX 
      parameters['active_BS'] = np.arange(1,self.active_BS + 1)
      # To activate the user rows 1-5 
      # according to practical examination i2_2p4 has 201 in each column 
      parameters['user_row_first'] = self.user_row_first
      parameters['user_row_last'] = self.user_row_last


      # Number of BS antennas in (x, y, z)
      parameters['bs_antenna']['shape'] = np.array([1, 1, 1])
      # To generate channels at 0.02 GHz  bandwidth, set
      parameters['OFDM']['bandwidth'] = self.bandwidth
      # To generate OFDM channels with 64 subcarriers, set
      parameters['OFDM']['subcarriers'] = self.subcarriers
      # To sample first 16 subcarriers by every spacing between each, set
      parameters['OFDM']['subcarriers_sampling'] = self.subcarriers_sampling
      #according to the picture in the article we chioced 32 instead of 16, that was mentioned in TABLE 1
      parameters['OFDM']['subcarriers_limit'] = self.subcarriers_limit
      # To only include 1 strongest paths in the channel computation, set
      parameters['num_paths'] = self.num_paths

      #Note: Since this scenario consists 
      #of only one access point, please set “enable_BS2BSchannels”
      #(MATLAB) or “enable_BS2BS” (Python) to zero in the DeepMIMO 
      #generation parameters. 
      parameters['enable_BS2BS'] = 0

      # Generate data
      dataset = DeepMIMO.generate_data(parameters)

      #delete unnccessary info
        #find keys in dictionary which is unneccessary
      dataset_keys = list(dataset[0].keys())
      dataset_user_keys = list(dataset[0]['user'].keys())
        #delte those keys 
      for i in range(len(dataset)):
          for del_key in [key for key in dataset_keys if key is not 'user']:
              del dataset[i][del_key]
          for del_key in [key for key in dataset_user_keys if key is not 'channel']:
              del dataset[i]['user'][del_key]

      return dataset

mim.y_train.shape

## PE

In [8]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [48]:
from generate_model import Designed_model as GM
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Reshape,Dense, GaussianNoise,Lambda,Dropout
from keras.models import Model
from keras import regularizers
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras import layers , Input
from tensorflow.keras.layers import Dense , LSTM ,Dropout , Activation ,Reshape

class PE(GM):
   def __init__(self,data_path):
      super(PE, self).__init__(data_path)
      self.name = 'pe'


   def build_model_and_get_data(self):
      if type(self.X_train) == type(None) : self.set_data() 

      self.model = self.RNN_model ( self.X_train[0].shape, 
                                    self.y_train[0].shape[0])
      
      self.compile_model(self.model)

      return 
   


   def train_gpu(self,epochs =20,batch_size=1024 ,compile = True): 
       with tf.device(tf.test.gpu_device_name()):
          self.train(epochs=epochs, batch_size=batch_size)    

   def train(self, epochs =20,batch_size=1024 ,compile = True, normalize = True , get_data_build =True):

      if get_data_build : self.build_model_and_get_data()
      if compile : self.model = self.compile_model(self.model)
      if normalize : self.norm ()

      history = self.model.fit(self.X_train,self.y_train,
              validation_data=(self.X_test, self.y_test),
              epochs=epochs, batch_size = batch_size, verbose=2 )
      
      return history
   
   def norm(self):

      self._mean = self.mean_data(np.array([self.mean_data(self.X_train),
                                          self.mean_data(self.X_test)]))

      self._scale = max(self.scale(self.X_train),
                        self.scale(self.X_test))
      
      for value in [self.X_train,self.X_test]:
        value = self.transform(value,self._mean,self._scale)

   
   
   
   def set_data(self , limit = 100000 , execute = False)  -> None:

      train_batch_size = 128
      train_snr = np.arange(20, -4, -2)
      test_snr = np.arange(0, 6.5, 0.5)
      train_ratio = np.array([0.4, 0.6, 0.8, 1.0])
      epoch_setting = np.array([10**1, 10**2, 10**3, 10**4, 10**5])

      if  execute : 
        os.mkdir(self.data_path)
        exec(open(self.data_path + 'deep-neural-network-decoder/RNN/noise/K_16_N_32/"train_data_10^6"/get_data.py'))

        ##### execute train ##########
        X_train, y_train = self.get_data_by_ratio (train_ratio = 0.8 ,
                                                   snr = train_snr,status = "train",
                                                   limit= limit,
                                                   data_path = self.data_path + "data/")    
        
        np.savez_compressed( self.data_path + 'train_shape', X_train_shape= np.array(X_train.shape) , y_train_shape = np.array(y_train.shape))
        print("train data shape saved")
        np.savez_compressed( self.data_path + 'train', X_train= X_train.reshape(-1,1), y_train = y_train.reshape(-1,1))
        print("train data saved")
        del X_train, y_train

        ##### execute test ##########
        X_test, y_test = self.get_data_by_ratio (train_ratio = 0.8 ,
                                                   snr = train_snr,status = "train",
                                                   limit= int(limit*0.2),
                                                   data_path = self.data_path + "data/",
                                                 ) 

        np.savez_compressed( self.data_path + 'test_shape', X_test_shape= np.array(X_test.shape) , y_test_shape = np.array(y_test.shape))
        print("test data shape saved")
        np.savez_compressed( self.data_path + 'test', X_test= X_test.reshape(-1,1), y_test = y_test.reshape(-1,1))
        print("test data saved")
        del X_test, y_test

      ###### Load data from npy saved based on exceuting data earlier ########
      print("Loading Train Data...")
      loaded = np.load( self.data_path + 'train.npz')
      loaded_shape = np.load( self.data_path + 'train_shape.npz')
      X_train_shape , y_train_shape = loaded_shape["X_train_shape"] , loaded_shape["y_train_shape"] 
      self.X_train , self.y_train = loaded["X_train"].reshape(X_train_shape) , loaded["y_train"].reshape(y_train_shape)


      print("Loading test Data...")
      loaded = np.load( self.data_path + 'test.npz')
      loaded_shape = np.load( self.data_path + 'test_shape.npz')
      X_test_shape , y_test_shape = loaded_shape["X_test_shape"] , loaded_shape["y_test_shape"] 
      self.X_test , self.y_test = loaded["X_test"].reshape(X_test_shape) , loaded["y_test"].reshape(y_test_shape)
      print("loading is finished ")
      del loaded
      # make the data independent of snr 
      self.X_train , self.y_train = self.X_train.reshape(-1,X_train_shape[-1]) , self.y_train.reshape(-1,y_train_shape[-1])
      self.X_test , self.y_test = self.X_test.reshape(-1,X_test_shape[-1]) , self.y_test.reshape(-1,y_test_shape[-1])
      
   
   
   @staticmethod
   def transform(x,mean,scale):
     return (x-mean)/scale

   @staticmethod
   def mean_data(data):
      mean = np.mean(data,axis=0)
      for i in range(1,len(data.shape)-2):
        mean = np.mean(mean,axis=0)   
      return mean

   @staticmethod 
   def scale(data):
      max = np.max(np.abs(data))
      return max 



   @staticmethod
   def compile_model(model):
      model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=0.001),
                    loss= tf.keras.losses.MeanSquaredError(),
                    metrics=["accuracy"],
                )
      return model
   

   @staticmethod
   def RNN_model (input_shape,output_shape, Dropout_rate = 0.1) :
  
    input_signal = Input(shape = input_shape)
    reshape = Reshape((*input_shape,1))(input_signal)
    lstm = LSTM(256,
                dropout = Dropout_rate,
                kernel_initializer= tf.keras.initializers.GlorotNormal(seed=None))(reshape)

    output = Dense(output_shape,"sigmoid")(lstm)
    
    return Model(inputs = input_signal, outputs = output)


   @staticmethod
   def get_data_by_ratio (train_ratio, snr, status = "train", limit= None,
                      data_path = "/content/project_3/PE/data/"):
      
      if status == "train" : data = sio.loadmat(data_path+"ratio_0.4_train_snr_8dB")
      elif status == "test" : data = sio.loadmat(data_path+'test_snr_0.0dB')
      x_shape = data['x_'+status].shape
      y_shape = data['y_'+status].shape
      x_shape = [min(x_shape[0],limit) , x_shape[1]]
      y_shape = [min(y_shape[0],limit) , y_shape[1]]

      x = np.zeros([len(snr),*x_shape])
      y = np.zeros([len(snr),*y_shape])
      print(x_shape)
      del data
    
      for i, tr_snr in enumerate(snr):
          print("added {} SNR".format(tr_snr))
          if status == "train" : filename = 'ratio_' + str(train_ratio) + '_train_snr_' + str(tr_snr) + 'dB'
          elif status == "test" : filename = 'test_snr_' + str(tr_snr) + 'dB'
          data = sio.loadmat(data_path+filename)
          x[i,:,:] = data['x_'+status][:limit,:]
          y[i,:,:] = data['y_'+status][:limit,:]

      return [x , y]






## MIST

In [133]:
import tensorflow as tf
import numpy as np
from keras.layers.core import Dense
from keras.layers import Conv1D,Flatten, BatchNormalization, Input
from keras import backend as K
from keras.models import Model
from keras.callbacks import ModelCheckpoint


from generate_model import Designed_model as GM
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Reshape,Dense, GaussianNoise,Lambda,Dropout
from keras.models import Model
from keras import regularizers
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

class MIST():
   def __init__(self, k = 500 ,
                      rate = 1/2 ,
                      SNR_dB_start_Eb = -1 ,
                      SNR_dB_stop_Eb = 7 ,
                      SNR_points = 9 ,
                      n_samples = 10 , 
                      train_batch = 1024,
                      test_batch = 1024,
                      g1=[1,1,1],
                      g2=[1,0,1]):
     
      #dataword length  => k  
      #encoder rate is used => rate 
      self.N = int(k/rate)  #codeword length
      for i,(key,value) in enumerate(locals().items()):
          if i < 1 : continue
          setattr(self,key,value)
      self.name = "mist"

   def build_model(self):
      self.model = self.MIST_model(input_shape=(self.N,1),output_shape=(self.k)) 
      return self.model

   def build_model_and_get_data(self):
      self.build_model()
      self.X_train , self.y_train = self.get_data()
      self.compile_model() 
   
   def train_gpu(self,epochs = 200 , batch_size=1024 ,compile = True): 
       with tf.device(tf.test.gpu_device_name()):
          self.train(epochs=epochs, batch_size=batch_size)    
 
   def train(self, epochs = 200, batch_size=1024 ,MIST_train = True ,compile = True, get_data_build =True):
      self.n_samples = int(epochs/self.SNR_points) 
      self.build_model()
      self.compile_model()
      
      if MIST_train : 
        history = self.MIST_train_idea()

      
      else :
          X_train , y_train = self.get_data()
          history = self.model.fit(X_train,y_train,
                epochs = 1, batch_size = batch_size, verbose=2 )
      return history
   
   
   def get_data(self , limit = None ) :
      sigmas = self. get_sigmas(self.SNR_dB_start_Eb,
                                self.SNR_dB_stop_Eb,
                                self.SNR_points,
                                self.k,
                                self.N)
      
      X,y = self.generate_uncode_noisy_signal(sigmas)
      return  [X,y]

  
   

   
   def generate_uncode_noisy_signal(self,sigmas):
     
     X = np.zeros([len(sigmas),self.n_samples,self.train_batch,self.N,1])
     y = np.zeros([len(sigmas),self.n_samples,self.train_batch,self.k])
     for i in range(0,self.n_samples): 
        for ii in range(0,len(sigmas)):
            #Generating dataword and codeword
            uncoded = np.random.randint(0,2,size=(self.train_batch,self.k))
            encoded = np.zeros([uncoded.shape[0], self.N])
            # memory 2 convolutional encoder used g1, g2
            for iii in range(0,self.train_batch): 
                encoded[iii,:] = self.convenc (uncoded[iii,:],self.g1,self.g2,self.k)
            #Modulate
            signal = 2*encoded - 1
            #Adding noise
            noisy_signal = signal + np.random.normal(0, sigmas[i], size= np.shape(signal))
            ## train 
            X[i,ii,:,:,0] = noisy_signal
            y[i,ii,:,:] = uncoded
      
     return [ X.reshape(-1,self.N) , y.reshape(-1,self.k)]




   def MIST_train_idea(self , limit = None ):
     sigmas = self.get_sigmas(self.SNR_dB_start_Eb,
                                self.SNR_dB_stop_Eb,
                                self.SNR_points,
                                self.k,
                                self.N)
     history = []
     self.X_train = np.zeros([len(sigmas),self.n_samples,self.train_batch,self.N,1])
     self.y_train = np.zeros([len(sigmas),self.n_samples,self.train_batch,self.k])
     for i in range(0,len(sigmas)):
       print("SNR  {}/{}".format(i,len(sigmas)))
       for ii in range(0,self.n_samples):
            #Generating dataword and codeword
            uncoded = np.random.randint(0,2,size=(self.train_batch,self.k))
            encoded = np.zeros([uncoded.shape[0], self.N])
            # memory 2 convolutional encoder used g1, g2
            for iii in range(0,self.train_batch): 
                encoded[iii,:] = self.convenc(uncoded[iii,:],self.g1,self.g2,self.k)
            #Modulate
            signal = 2*encoded - 1
            #Adding noise
            noisy_signal = signal + np.random.normal(0, sigmas[i], size= np.shape(signal))
            ## train l
            print("epoch  {}/{}".format(ii,self.n_samples))
            self.X_train[i,ii,:,:,0] = noisy_signal
            self.y_train[i,ii,:,:] = uncoded
            x_train = np.expand_dims(noisy_signal,axis=2)
            history.append( self.model.fit(x_train,uncoded, epochs = 1, batch_size = self.train_batch , verbose=2))

     return history 

   def compile_model(self, optimizer = 'adam',loss = 'mse' ):            
      self.model.compile(optimizer=optimizer, loss=loss, metrics=[self.ber]) 
      return self.model

   
   @staticmethod
   def get_sigmas(SNR_dB_start_Eb,SNR_dB_stop_Eb,SNR_points,k,N):
      
      SNR_dB_start_Es = SNR_dB_start_Eb + (10*np.log10(1.0*k/N))
      SNR_dB_stop_Es = SNR_dB_stop_Eb + (10*np.log10(1.0*k/N))
      SNR_range=np.linspace(SNR_dB_start_Es, SNR_dB_stop_Es, SNR_points)
      return np.sqrt(1/(2*10**(SNR_range/10)))
   
   
   @staticmethod
   def convenc(data,g1,g2,k):
    enc_msg = np.zeros([2*k])
    enc_msg[0::2] = (np.convolve(data,g1)%2)[0:k]
    enc_msg[1::2] = (np.convolve(data,g2)%2)[0:k]
    return enc_msg   

    
   @staticmethod
   def ber(y_true, y_pred):
     return  K.mean(K.cast(K.not_equal(y_true, K.round(y_pred)),dtype='float32'))
   
   @staticmethod
   def MIST_model(input_shape , output_shape):  
      input_batch=Input(shape=input_shape)
      conv1 = Conv1D(10, 24, activation='relu',padding='same')(input_batch)
      batch_norm1=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv1)
      conv2 = Conv1D(50, 24, activation='relu',padding='same')(batch_norm1)
      batch_norm2=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv2)
      conv3 = Conv1D(50, 24, activation='relu',padding='same')(batch_norm2)
      batch_norm3=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv3)
      flatten = Flatten()(batch_norm3)
      msg_out = Dense(output_shape, activation='sigmoid')(flatten)

      return Model(inputs=input_batch,outputs=msg_out)
  

# test

In [12]:
a = aphl("/content/gdrive/MyDrive/projects/RML2016.10a/")

In [13]:
p = PE("/content/gdrive/MyDrive/projects/project_3/PE")

In [100]:
mim = Deep_MIMO('/content/gdrive/MyDrive/projects/DeepMIMO2',active_BS=20,user_row_last=2)

In [120]:
mis = MIST()

In [ ]:
a.train_gpu(epochs=20)

In [101]:
mim.train()


Basestation 1

UE-BS Channels


Generating channels: 100%|██████████| 402/402 [00:00<00:00, 6317.15it/s]



Basestation 2

UE-BS Channels


Generating channels: 100%|██████████| 402/402 [00:00<00:00, 5997.13it/s]



Basestation 3

UE-BS Channels


...


Basestation 19

UE-BS Channels


Generating channels: 100%|██████████| 402/402 [00:00<00:00, 5977.57it/s]



Basestation 20

UE-BS Channels


Generating channels: 100%|██████████| 402/402 [00:00<00:00, 6223.99it/s]


(20, 402, 32)
(20, 402, 32)
Epoch 1/17
3/3 - 1s - loss: 0.3860 - accuracy: 0.5429 - val_loss: 0.3507 - val_accuracy: 0.6199 - 1s/epoch - 344ms/step
Epoch 2/17
3/3 - 0s - loss: 0.3032 - accuracy: 0.6771 - val_loss: 0.2440 - val_accuracy: 0.7380 - 183ms/epoch - 61ms/step
Epoch 3/17
3/3 - 0s - loss: 0.1800 - accuracy: 0.8063 - val_loss: 0.1778 - val_accuracy: 0.8064 - 171ms/epoch - 57ms/step
Epoch 4/17
3/3 - 0s - loss: 0.1047 - accuracy: 0.8795 - val_loss: 0.1280 - val_accuracy: 0.8605 - 161ms/epoch - 54ms/step
Epoch 5/17
3/3 - 0s - loss: 0.0628 - accuracy: 0.9181 - val_loss: 0.1090 - val_accuracy: 0.8888 - 151ms/epoch - 50ms/step
Epoch 6/17
3/3 - 0s - loss: 0.0418 - accuracy: 0.9371 - val_loss: 0.0842 - val_accuracy: 0.9135 - 154ms/epoch - 51ms/step
Epoch 7/17
3/3 - 0s - loss: 0.0315 - accuracy: 0.9481 - val_loss: 0.1022 - val_accuracy: 0.9141 - 147ms/epoch - 49ms/step
Epoch 8/17
3/3 - 0s - loss: 0.0284 - accuracy: 0.9540 - val_loss: 0.0802 - val_accuracy: 0.9249 - 153ms/epoch - 51ms/ste

In [102]:
p.train_gpu()

Loading Train Data...
Loading test Data...
loading is finished 
Epoch 1/20
5860/5860 - 332s - loss: 0.0993 - accuracy: 0.1630 - val_loss: 0.0653 - val_accuracy: 0.1129 - 332s/epoch - 57ms/step
Epoch 2/20
5860/5860 - 308s - loss: 0.0638 - accuracy: 0.1324 - val_loss: 0.0542 - val_accuracy: 0.1393 - 308s/epoch - 53ms/step
Epoch 3/20


KeyboardInterrupt: ignored

In [103]:
mim.train_gpu()

Epoch 1/17
3/3 - 1s - loss: 0.3886 - accuracy: 0.5409 - val_loss: 0.3522 - val_accuracy: 0.6111 - 988ms/epoch - 329ms/step
Epoch 2/17
3/3 - 0s - loss: 0.3117 - accuracy: 0.6612 - val_loss: 0.2598 - val_accuracy: 0.7237 - 167ms/epoch - 56ms/step
Epoch 3/17
3/3 - 0s - loss: 0.1968 - accuracy: 0.7908 - val_loss: 0.1873 - val_accuracy: 0.7993 - 159ms/epoch - 53ms/step
Epoch 4/17
3/3 - 0s - loss: 0.1182 - accuracy: 0.8661 - val_loss: 0.1386 - val_accuracy: 0.8500 - 197ms/epoch - 66ms/step
Epoch 5/17
3/3 - 0s - loss: 0.0723 - accuracy: 0.9089 - val_loss: 0.1117 - val_accuracy: 0.8829 - 147ms/epoch - 49ms/step
Epoch 6/17
3/3 - 0s - loss: 0.0471 - accuracy: 0.9319 - val_loss: 0.0950 - val_accuracy: 0.9041 - 145ms/epoch - 48ms/step
Epoch 7/17
3/3 - 0s - loss: 0.0355 - accuracy: 0.9442 - val_loss: 0.0993 - val_accuracy: 0.9137 - 145ms/epoch - 48ms/step
Epoch 8/17
3/3 - 0s - loss: 0.0315 - accuracy: 0.9499 - val_loss: 0.0879 - val_accuracy: 0.9196 - 141ms/epoch - 47ms/step
Epoch 9/17
3/3 - 0s - l

In [122]:
mis.train()

SNR  0/9
epoch  0/22
1/1 - 6s - loss: 0.3179 - ber: 0.5004 - 6s/epoch - 6s/step
epoch  1/22
1/1 - 1s - loss: 0.3186 - ber: 0.5000 - 565ms/epoch - 565ms/step
epoch  2/22
1/1 - 1s - loss: 0.3206 - ber: 0.4974 - 559ms/epoch - 559ms/step
epoch  3/22
1/1 - 1s - loss: 0.3241 - ber: 0.4970 - 562ms/epoch - 562ms/step
epoch  4/22
1/1 - 1s - loss: 0.3263 - ber: 0.4953 - 556ms/epoch - 556ms/step
epoch  5/22
1/1 - 1s - loss: 0.3282 - ber: 0.4949 - 567ms/epoch - 567ms/step
epoch  6/22
1/1 - 1s - loss: 0.3288 - ber: 0.4939 - 568ms/epoch - 568ms/step
epoch  7/22
1/1 - 1s - loss: 0.3278 - ber: 0.4931 - 560ms/epoch - 560ms/step
epoch  8/22
1/1 - 1s - loss: 0.3270 - ber: 0.4916 - 576ms/epoch - 576ms/step
epoch  9/22
1/1 - 1s - loss: 0.3250 - ber: 0.4905 - 557ms/epoch - 557ms/step
epoch  10/22
1/1 - 1s - loss: 0.3230 - ber: 0.4889 - 566ms/epoch - 566ms/step
epoch  11/22
1/1 - 1s - loss: 0.3225 - ber: 0.4888 - 554ms/epoch - 554ms/step
epoch  12/22
1/1 - 1s - loss: 0.3211 - ber: 0.4878 - 569ms/epoch - 569m

In [111]:
def show (m):
  print('train shape : {}'.format(m.X_train.shape))
  print('first train y {}'.format(m.y_train[0]))
  print("model {}".format(m.model.summary()))  
  print("the model is  {}".format(m.name))

In [112]:
show(a)

train shape : (132000, 2, 128)
first train y [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2, 128)]          0         
                                                                 
 reshape_1 (Reshape)         (None, 2, 128, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 121, 128)       2176      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 60, 128)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 1, 45, 64)         131136    
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 22, 6

AttributeError: ignored

In [109]:
show(p)

train shape : (6000000, 32)
first train y [0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1.]
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32)]              0         
                                                                 
 reshape_6 (Reshape)         (None, 32, 1)             0         
                                                                 
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense_24 (Dense)            (None, 16)                4112      
                                                                 
Total params: 268,304
Trainable params: 268,304
Non-trainable params: 0
_________________________________________________________________
model None


In [110]:
show(mim)

train shape : (269, 20, 32, 2)
first train y [[[-0.40388584  0.59408945]
  [-0.3435466   0.62768018]
  [-0.28005663  0.65484971]
  ...
  [ 0.37792793 -0.48241663]
  [ 0.32272899 -0.52391654]
  [ 0.26353374 -0.55948448]]

 [[-0.04350499  0.7364549 ]
  [ 0.02544204  0.73246753]
  [ 0.09367171  0.72177613]
  ...
  [ 0.15945612 -0.64666349]
  [ 0.09256665 -0.66385049]
  [ 0.02432002 -0.67443305]]

 [[ 0.69906861 -0.01982092]
  [ 0.69140226 -0.08845964]
  [ 0.67745835 -0.15610296]
  ...
  [-0.73911095 -0.2677696 ]
  [-0.76348132 -0.20314656]
  [-0.7818076  -0.1365568 ]]

 ...

 [[ 0.66550201 -0.0382316 ]
  [ 0.6557349  -0.1066007 ]
  [ 0.63944864 -0.17371619]
  ...
  [-0.74432439 -0.15502647]
  [-0.7587921  -0.08749562]
  [-0.76670915 -0.01888767]]

 [[ 0.05414769 -0.72757643]
  [-0.01463112 -0.73387527]
  [-0.08369773 -0.73393506]
  ...
  [-0.51495934  0.63486946]
  [-0.45833212  0.6744113 ]
  [-0.39836434  0.70867592]]

 [[-0.56452352  0.65610486]
  [-0.50931966  0.69761503]
  [-0.4507848

In [124]:
show(mis)

train shape : (9, 22, 1024, 1000, 1)
first train y [[[0. 1. 0. ... 1. 1. 1.]
  [0. 0. 1. ... 1. 1. 0.]
  [1. 0. 1. ... 1. 1. 0.]
  ...
  [1. 0. 0. ... 1. 0. 1.]
  [0. 1. 0. ... 1. 0. 0.]
  [0. 1. 1. ... 1. 0. 1.]]

 [[1. 1. 0. ... 0. 1. 1.]
  [0. 1. 0. ... 1. 1. 0.]
  [0. 1. 0. ... 1. 1. 1.]
  ...
  [1. 0. 1. ... 0. 1. 1.]
  [1. 0. 0. ... 1. 0. 0.]
  [1. 0. 1. ... 1. 1. 1.]]

 [[1. 0. 1. ... 0. 0. 0.]
  [0. 0. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 0. 1. 1.]
  ...
  [1. 1. 1. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [1. 1. 0. ... 0. 1. 1.]]

 ...

 [[0. 1. 0. ... 1. 0. 1.]
  [0. 1. 1. ... 0. 1. 1.]
  [1. 1. 0. ... 1. 0. 1.]
  ...
  [1. 0. 1. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [1. 1. 0. ... 0. 1. 0.]]

 [[1. 0. 1. ... 1. 1. 0.]
  [1. 0. 1. ... 1. 0. 1.]
  [0. 1. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 1. 1. 0.]
  [1. 1. 0. ... 0. 0. 1.]
  [1. 1. 1. ... 1. 0. 0.]]

 [[0. 0. 0. ... 1. 0. 1.]
  [0. 1. 0. ... 1. 0. 1.]
  [0. 1. 1. ... 0. 0. 0.]
  ...
  [1. 1. 0. ... 0. 1. 0.]
  [0. 1. 1. .

## get clean data

In [128]:
def get_clean_data(a,limit=100):
  pred = a.model.predict(a.X_train)
  if a.name == "deepmimo":
    diff = np.mean(np.mean(np.mean(np.abs(pred - a.y_train),axis=-1),axis=-1),axis=-1)
  else : diff = np.mean(np.abs(pred - a.y_train),axis=-1)
  best_answer_index = np.argsort(diff)
  return dict(X_train = a.X_train[best_answer_index][:limit] , y_train = a.y_train[best_answer_index][:limit])

In [130]:
clean = get_clean_data(p)

AttributeError: ignored

In [93]:
a.model.evaluate(clean["X_train"],clean["y_train"])

4/4 [==============================] - 0s 102ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

#draft 

## foolbox

In [ ]:
!pip3 install foolbox==3.1.1
# !pip3 install git+https://github.com/bethgelab/foolbox.git

In [50]:
!pip3 install --upgrade tensorflow

In [ ]:
show(a)

In [82]:
import foolbox as fb
model = a.model
preprocessing = dict()
bounds = (np.min(a.X_train), np.max(a.X_train))
fmodel = fb.TensorFlowModel(a.model, bounds=bounds,preprocessing=preprocessing)
fb.utils.accuracy(fmodel, a.X_train[:17], a.y_train[:17])
attack = fb.attacks.LinfDeepFoolAttack()
raw, clipped, is_adv = attack(fmodel, a.X_train[:40], a.y_train[:40], epsilons=0.03)

NotImplementedError: ignored

In [57]:
a.model.evaluate(a.X_train[:16],a.y_train[:16])

1/1 [==============================] - 0s 366ms/step - loss: 2.3979 - accuracy: 0.0625


[2.3978676795959473, 0.0625]

In [26]:
i = tf.convert_to_tensor(a.X_train, np.float64)

In [98]:
model = tf.keras.applications.ResNet50(weights="imagenet")
model

In [18]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

# Use Pandas to load dataset from csv file
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001
y_train = y_train.reshape(-1,1)

inputs = keras.Input(shape=(x_train[0].shape))
x = layers.Dense(128, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)
output1 = layers.Dense(2, name="first_num")(x)
model = keras.Model(inputs=inputs, outputs=output1)

model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss='mse',
    metrics=["accuracy"],
)

model.fit(x_train,y_train, epochs=5, verbose=2)
model.evaluate(x_test,y_test, verbose=2)

Epoch 1/5
1875/1875 - 5s - loss: 1.7354 - accuracy: 0.1005 - 5s/epoch - 3ms/step
Epoch 2/5
1875/1875 - 7s - loss: 0.7782 - accuracy: 0.1062 - 7s/epoch - 4ms/step
Epoch 3/5
1875/1875 - 7s - loss: 0.5569 - accuracy: 0.1052 - 7s/epoch - 4ms/step
Epoch 4/5
1875/1875 - 8s - loss: 0.4338 - accuracy: 0.1030 - 8s/epoch - 4ms/step
Epoch 5/5
1875/1875 - 8s - loss: 0.3503 - accuracy: 0.1028 - 8s/epoch - 4ms/step
313/313 - 1s - loss: 0.5012 - accuracy: 0.1371 - 1s/epoch - 3ms/step


[0.5012049078941345, 0.1370999962091446]

## advbox

In [28]:
!pip install advbox 

  Using cached advbox-0.4.0.tar.gz (708 kB)
ERROR: Could not find a version that satisfies the requirement advbox (from versions: 0.4.0)
ERROR: No matching distribution found for advbox


In [27]:
from adversarialbox.adversary import Adversary

ModuleNotFoundError: ignored

In [30]:
%mkdir /content/gdrive/MyDrive/attack

In [31]:
cd /content/gdrive/MyDrive/attack

/content/gdrive/MyDrive/attack


In [34]:
!git clone https://github.com/baidu/AdvBox.git  

Cloning into 'AdvBox'...
remote: Enumerating objects: 2209, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2209 (delta 16), reused 12 (delta 7), pack-reused 2185
Receiving objects: 100% (2209/2209), 101.65 MiB | 11.12 MiB/s, done.
Resolving deltas: 100% (1363/1363), done.
Checking out files: 100% (226/226), done.


In [35]:
cd /content/gdrive/MyDrive/attack/AdvBox/

/content/gdrive/MyDrive/attack/AdvBox


In [36]:
!pip install -r requirements.txt

     |████████████████████████████████| 88 kB 3.4 MB/s 
     |████████████████████████████████| 147 kB 36.8 MB/s 
ERROR: Could not find a version that satisfies the requirement futures==3.2.0 (from versions: 0.2.python3, 0.1, 0.2, 1.0, 2.0, 2.1, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.1.5, 2.1.6, 2.2.0, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.0.4, 3.0.5, 3.1.0, 3.1.1)
ERROR: No matching distribution found for futures==3.2.0


In [37]:
import sys
sys.path.append('/content/gdrive/MyDrive/attack/AdvBox/adversarialbox')

https://github.com/advboxes/AdvBox/blob/master/ebook/ebook_imagenet_fgsm_tf.ipynb

In [40]:
from adversarialbox.adversary import Adversary
from adversarialbox.attacks.gradient_method import FGSM
from adversarialbox.models.keras import KerasModel
from adversarialbox.attacks.deepfool import DeepFoolAttack
from adversarialbox.models.tensorflow import TensorflowModel

In [50]:
keras.backend.set_learning_phase(0)
attack = FGSM(a.model)
attack_config = {"epsilons": 1, "epsilons_max": 10, "epsilon_steps": 1, "steps": 100}
adversary = Adversary(a.X_train,None)
adversary = attack(adversary, **attack_config)
if adversary.is_successful():
        print('attack success')

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


AttributeError: ignored

In [ ]:


session = tf.Session()
session.run(tf.global_variables_initializer())
tensorlist=[n.name for n in session.graph_def.node]
m = TensorflowModel(
    session,
    x,
    None,
    logits,
    None,
    bounds=(-1, 1),
    channel_axis = 3,
    preprocess=None)


attack = DeepFoolAttack(m)
attack_config = {"iterations": 100, "overshoot": 0.05}



In [51]:
!pip install pillow

In [56]:
a.model.save('/content/gdrive/MyDrive/proj/my.pb')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/proj/my.pb/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/proj/my.pb/assets


In [59]:

#[224,224,3]->[1,224,224,3]
image=a.X_train[0]

session= tf.compat.v1.Session()

def create_graph(dirname):
    with tf.compat.v1.gfile.FastGFile(dirname, 'rb') as f:
        graph_def = session.graph_def
        graph_def.ParseFromString(f.read())

        _ = tf.import_graph_def(graph_def, name='')

create_graph("/content/gdrive/MyDrive/proj/my.pb/saved_model.pb")

# 初始化参数  非常重要
session.run(tf.global_variables_initializer())

tensorlist=[n.name for n in session.graph_def.node]

#输出全部tensor
#logger.info(tensorlist)


#获取logits
logits=session.graph.get_tensor_by_name('softmax/logits:0')

x = session.graph.get_tensor_by_name('ExpandDims:0')

# advbox demo
# 因为原始数据没有归一化  所以bounds=(0, 255)
m = TensorflowModel(
    session,
    x,
    None,
    logits,
    None,
    bounds=(0, 255),
    channel_axis=3,
    preprocess=None)

#实例化DeepFool 进行定向攻击 
attack = DeepFoolAttack(m)
attack_config = {"iterations": 100, "overshoot": 0.05}

adversary = Adversary(image,None)
#麦克风
tlabel = 651
adversary.set_target(is_targeted_attack=True, target_label=tlabel)

# FGSM targeted attack
adversary = attack(adversary, **attack_config)

if adversary.is_successful():
    print(
        'attack success, adversarial_label=%d'
        % (adversary.adversarial_label) )

    #对抗样本保存在adversary.adversarial_example
    adversary_image=np.copy(adversary.adversarial_example)
    #强制类型转换 之前是float 现在要转换成int8

    adv = np.array(adversary_image).astype("uint8")[0]

 

print("DeepFool attack done")

AttributeError: ignored

In [60]:
%mkdir "/content/gdrive/MyDrive/attack"

In [61]:
%cd "/content/gdrive/MyDrive/attack"

/content/gdrive/MyDrive/attack


## deepsec

In [62]:
#!git clone https://github.com/ryderling/DEEPSEC

Cloning into 'DEEPSEC'...
remote: Enumerating objects: 137, done.
remote: Total 137 (delta 0), reused 0 (delta 0), pack-reused 137
Receiving objects: 100% (137/137), 175.67 MiB | 13.08 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Checking out files: 100% (101/101), done.


In [15]:
import sys
sys.path.append('/content/gdrive/MyDrive/attack/DEEPSEC')

In [81]:
!python /content/gdrive/MyDrive/attack/DEEPSEC/Attacks/FGSM_Generation.py --dataset=mnist --epsilon=0.3

../RawModels/MNIST/model/MNIST_raw.pt
starting to LOAD the $<class 'RawModels.MNISTConv.MNISTConvNet'>$ Model from ../RawModels/MNIST/model/MNIST_raw.pt within the cpu device
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/attack/DEEPSEC/Attacks/FGSM_Generation.py", line 93, in <module>
    main(arguments)
  File "/content/gdrive/MyDrive/attack/DEEPSEC/Attacks/FGSM_Generation.py", line 72, in main
    attack_batch_size=args.attack_batch_size)
  File "/content/gdrive/MyDrive/attack/DEEPSEC/Attacks/FGSM_Generation.py", line 27, in __init__
    super(FGSMGeneration, self).__init__(dataset, attack_name, targeted, raw_model_location, clean_data_location, adv_examples_dir, device)
  File "/content/gdrive/MyDrive/attack/DEEPSEC/Attacks/../Attacks/Generation.py", line 44, in __init__
    self.raw_model.load(path=raw_model_location, device=device)
  File "/content/gdrive/MyDrive/attack/DEEPSEC/Attacks/../RawModels/basic_module.py", line 33, in load
    self.load_state_dict(to

In [16]:
from Attacks.AttackMethods.AttackUtils import tensor2variable

In [23]:
a.train()

Epoch 1/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 193s - loss: 2.0789 - accuracy: 0.2305 - val_loss: 1.8752 - val_accuracy: 0.2945 - 193s/epoch - 1s/step
Epoch 2/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 182s - loss: 1.7964 - accuracy: 0.3201 - val_loss: 1.7380 - val_accuracy: 0.3410 - 182s/epoch - 1s/step
Epoch 3/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 182s - loss: 1.6931 - accuracy: 0.3583 - val_loss: 1.6550 - val_accuracy: 0.3674 - 182s/epoch - 1s/step
Epoch 4/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 180s - loss: 1.6251 - accuracy: 0.3781 - val_loss: 1.6035 - val_accuracy: 0.3811 - 180s/epoch - 1s/step
Epoch 5/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 180s - loss: 1.5779 - accuracy: 0.3901 - val_loss: 1.5571 - val_accuracy: 0.3940 - 180s/e

In [20]:
data_path = '/content/gdrive/MyDrive/data/'
np.savez_compressed( data_path + 'train_shape', X_train_shape= np.array(a.X_train.shape) , y_train_shape = np.array(a.y_train.shape))
print("train data shape saved")
np.savez_compressed( data_path + 'train', X_train= a.X_train.reshape(-1,1), y_train = a.y_train.reshape(-1,1))
print("train data saved")

train data shape saved
train data saved


In [ ]:
loaded = np.load( self.data_path + 'train.npz')
loaded_shape = np.load( self.data_path + 'train_shape.npz')
X_train_shape , y_train_shape = loaded_shape["X_train_shape"] , loaded_shape["y_train_shape"] 

In [22]:
a.model.save("/content/gdrive/MyDrive/data/myModel.h5")
a.model.save_weights("/content/gdrive/MyDrive/data/weight.h5")

In [ ]:
from keras.models import load_model
b = load_model("myModel.h5")
new_model.load_weights("myModel.h5")

## cleverhans

In [ ]:
!pip install cleverhans

In [58]:
import subprocess
subprocess.call(['pip', 'install', "cleverhans"])
import math
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from easydict import EasyDict
from tensorflow.keras import Model
from tensorflow.keras.layers import AveragePooling2D, Conv2D

from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

def attack_handler(a , FLAGS = {"nb_epochs": 1,"eps": 0.05,"adv_train": False} ,batch = 128):

    train = [t[:128*1000].reshape(-1,batch,*t.shape[1:]) for t in [a.X_train,a.y_train]]
    test = [t[:128*200].reshape(-1,batch,*t.shape[1:]) for t in [a.X_test,a.y_test]]

    data = EasyDict(train=train, test=test)
    # Load training and test data
    model = a.model
    loss_object =tf.keras.losses.MeanSquaredError()
    optimizer = tf.optimizers.Adam(learning_rate=0.001)

    # Metrics to track the different accuracies.
    train_loss = tf.metrics.Mean(name="train_loss")
    test_acc = tf.keras.metrics.CategoricalAccuracy()
    test_acc_fgsm = tf.keras.metrics.CategoricalAccuracy()
    test_acc_pgd = tf.keras.metrics.CategoricalAccuracy()

    @tf.function
    def train_step(x, y):
        with tf.GradientTape() as tape:
            predictions = model(x)
            loss = loss_object(y, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)

    print("Train model with adversarial training")
    # Train model with adversarial training
    for epoch in range(FLAGS["nb_epochs"]):
        # keras like display of progress
        progress_bar_train = tf.keras.utils.Progbar(50000)
        for i in range(data.train[0].shape[0]):
            if FLAGS['adv_train']:
                # Replace clean example with adversarial example for adversarial training
                x = projected_gradient_descent(model, data.train[0][i,:], FLAGS['eps'], 0.01, 40, np.inf)
            train_step(data.train[0][i,:], data.train[1][i,:])
            progress_bar_train.add(data.train[0][i,:].shape[0], values=[("loss", train_loss.result())])

    print("Evaluate on clean and adversarial data")
    print("getting clean data from model")
    
    # Evaluate on clean and adversarial data
    def get_clean_data(a,limit=100):
          pred = a.model.predict(a.X_train)
          if a.name == "deepmimo":
            diff = np.mean(np.mean(np.mean(np.abs(pred - a.y_train),axis=-1),axis=-1),axis=-1)
          else : diff = np.mean(np.abs(pred - a.y_train),axis=-1)
          best_answer_index = np.argsort(diff)
          return dict(X = a.X_train[best_answer_index][:limit] , y = a.y_train[best_answer_index][:limit])

    clean = get_clean_data(a)

    progress_bar_test = tf.keras.utils.Progbar(10000)
    for i in range(clean["X"].shape[0]):
        y_pred = model(clean["X"][i,:])
        test_acc(clean["y"][i,:], y_pred)

        x_fgm = fast_gradient_method(model, clean["X"][i,:], FLAGS.eps, np.inf)
        y_pred_fgm = model(x_fgm)
        test_acc_fgsm(clean["y"][i,:], y_pred_fgm)

        x_pgd = projected_gradient_descent(model, clean["X"][i,:], FLAGS.eps, 0.01, 40, np.inf)
        y_pred_pgd = model(x_pgd)
        test_acc_pgd(clean["y"][i,:], y_pred_pgd)

        progress_bar_test.add(clean["X"][i,:].shape[0])
    
    print(a.name)    

    print(
        "test acc on clean examples (%): {:.3f}".format(test_acc.result() * 100)
    )
    print(
        "test acc on FGM adversarial examples (%): {:.3f}".format(
            test_acc_fgsm.result() * 100
        )
    )
    print(
        "test acc on PGD adversarial examples (%): {:.3f}".format(
            test_acc_pgd.result() * 100
        )
    )

    return dict(acc = test_acc.result(),acc_FGM = test_acc_fgsm.result(),acc_pdg = test_acc_pgd.result())



128000/50000 [============================================================================] - 195s 2ms/step - loss: 0.0351


AttributeError: ignored

In [61]:
FLAGS[eps

AttributeError: ignored

In [62]:

# Evaluate on clean and adversarial data
progress_bar_test = tf.keras.utils.Progbar(10000)
for i in range(data.test[0].shape[0]):
    y_pred = model(data.test[0][i,:])
    test_acc_clean(data.test[1][i,:], y_pred)

    x_fgm = fast_gradient_method(model, data.test[0][i,:], FLAGS['eps'], np.inf)
    y_pred_fgm = model(x_fgm)
    test_acc_fgsm(data.test[1][i,:], y_pred_fgm)

    x_pgd = projected_gradient_descent(model, data.test[0][i,:], FLAGS['eps'], 0.01, 40, np.inf)
    y_pred_pgd = model(x_pgd)
    test_acc_pgd(data.test[1][i,:], y_pred_pgd)

    progress_bar_test.add(data.test[0][i,:].shape[0])

print(
    "test acc on clean examples (%): {:.3f}".format(test_acc_clean.result() * 100)
)
print(
    "test acc on FGM adversarial examples (%): {:.3f}".format(
        test_acc_fgsm.result() * 100
    )
)
print(
    "test acc on PGD adversarial examples (%): {:.3f}".format(
        test_acc_pgd.result() * 100
    )
)

25600/10000 [============================================================================] - 1058s 41ms/step
test acc on clean examples (%): 13.032
test acc on FGM adversarial examples (%): 8.977
test acc on PGD adversarial examples (%): 10.961
